#### Change th

In [ ]:
import os
from PIL import Image

def convert_webp_to_jpg(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.webp'):
            # Construct the full input path
            input_path = os.path.join(input_folder, filename)
            # Construct the full output path (change extension to .jpg)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.jpg")

            # Open the WEBP image file
            with Image.open(input_path) as image:
                # Convert the image to RGB mode
                rgb_image = image.convert('RGB')
                # Save the file as a JPEG file
                rgb_image.save(output_path, 'JPEG')
            print(f"Converted {filename} to JPEG format.")



In [ ]:
# Example usage:
convert_webp_to_jpg('data/poor/img_seq4/renamed', 'data/poor/img_seq4/renamed')


In [ ]:
import os

# Specify the directory containing the image files
directory = 'data/poor/img_seq4/renamed'

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.startswith('img (') and filename.endswith(').jpg'):
        print(filename)
        # Extract the number from the filename
        number = int(filename.split('(')[1].split(')')[0])
        
        # Generate the new filename with leading zeros
        new_filename = f'img ({number:02}).jpg'
        
        # Rename the file
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

#### B. Enforcement (sample code)

In [ ]:
import random

def fairness_enforcement(items, beta):

    if beta < len(items):
        return "The length of the items should at least be as large as the beta value"
    
    # Initialize the counters for each item with the value beta
    counters = {item: beta for item in items}
    
    while True:
        # Display current counters for transparency
        print("Current counters:", counters)
        
        # Get user input and ensure it is within the list of items
        user_input = input("Enter an item from the list {} or type 'EXIT' to quit: ".format(items))
        
        # Check for exit condition
        if user_input.upper() == "EXIT":
            print("Exiting the program.")
            break
        
        # Ensure the input is within the list of items
        while user_input not in items:
            user_input = input("Please enter a valid item from the list {}, or type 'EXIT' to quit: ".format(items))
            if user_input.upper() == "EXIT":
                print("Exiting the program.")
                return
        
        # Check the condition from largest to smallest
        for k in range(beta, 0, -1):
            # Find items with a counter value of exactly k
            matching_items = [item for item, count in counters.items() if count == k]
            # If there are exactly k items with the counter value k
            if len(matching_items) == k:
                # Randomly select an item from those that match the condition
                replacement = random.choice(matching_items)
                final_input = replacement
                print("Replacing '{}' with '{}' based on the condition.".format(user_input, replacement))
                break
        else:
            # If no condition was met, the final input is the user's input
            final_input = user_input
        
        # Reset the counter of the final input to beta and decrement others by 1
        for item in counters:
            if item == final_input:
                counters[item] = beta
            else:
                counters[item] -= 1



In [ ]:
# Example usage
items = ["apple", "orange", "banana", "grape"]
beta = 6
fairness_enforcement(items, beta)

#### C. Pairwise coverage monitoring (sample code)

In [ ]:
import itertools

def generate_all_possible_pairs(categories):
    category_pairs = list(itertools.combinations(categories.keys(), 2))
    possible_pairs = {}
    for (cat1, cat2) in category_pairs:
        pairs = list(itertools.product(categories[cat1], categories[cat2]))
        possible_pairs[(cat1, cat2)] = set(pairs)
    return possible_pairs

def update_covered_pairs(test_case, covered_pairs, all_possible_pairs):
    for (cat1, cat2), pairs in all_possible_pairs.items():
        pair = (test_case[cat1], test_case[cat2])
        if pair in pairs:
            covered_pairs[(cat1, cat2)].add(pair)
    return covered_pairs

def suggest_test_case(categories, covered_pairs, all_possible_pairs):
    missing_pairs = {key: pairs - covered for key, pairs in all_possible_pairs.items() for covered in [covered_pairs[key]]}
    
    # Count total and missing pairs
    total_pairs = sum(len(pairs) for pairs in all_possible_pairs.values())
    total_missing_pairs = sum(len(pairs) for pairs in missing_pairs.values())
    
    # Create a dictionary to track the coverage improvement of each possible test case
    test_case_coverage = {}

    # Generate all possible combinations of test cases
    all_possible_test_cases = list(itertools.product(*[categories[key] for key in sorted(categories.keys())]))

    for test_case in all_possible_test_cases:
        test_case_dict = dict(zip(sorted(categories.keys()), test_case))
        # Count how many new pairs this test case would cover
        new_coverage_count = sum((test_case_dict[cat1], test_case_dict[cat2]) in missing_pairs[(cat1, cat2)] for (cat1, cat2) in missing_pairs)

        if new_coverage_count > test_case_coverage.get('max_coverage', 0):
            test_case_coverage['best_test_case'] = test_case
            test_case_coverage['max_coverage'] = new_coverage_count

    return {
        'suggested_test_case': test_case_coverage.get('best_test_case', 'All pairs are already covered!'),
        'total_missing_pairs': total_missing_pairs,
        'total_pairs': total_pairs
    }



In [ ]:
# Example usage
categories = {
    'A': ['a1', 'a2', 'a3'],
    'B': ['b1', 'b2', 'b3', 'b4'],
    'C': ['c1', 'c2'],
    'D': ['d1', 'd2', 'd3']
}

all_possible_pairs = generate_all_possible_pairs(categories)
covered_pairs = {key: set() for key in all_possible_pairs.keys()}



In [ ]:
# Simulate iterative user input
user_inputs = [
    {'A': 'a1', 'B': 'b2', 'C': 'c1', 'D': 'd2'},
    {'A': 'a2', 'B': 'b3', 'C': 'c2', 'D': 'd1'}
]

for user_input in user_inputs:
    covered_pairs = update_covered_pairs(user_input, covered_pairs, all_possible_pairs)

# Display suggestion and coverage status
suggestion = suggest_test_case(categories, covered_pairs, all_possible_pairs)
print("Suggested test case to maximally increase pair-wise coverage:", suggestion['suggested_test_case'])
print("Total missing pairs:", suggestion['total_missing_pairs'])
print("Total possible pairs:", suggestion['total_pairs'])


In [ ]:
# Simulate iterative user input
user_inputs = [
    {'A': 'a2', 'B': 'b4', 'C': 'c1', 'D': 'd3'}
]

for user_input in user_inputs:
    covered_pairs = update_covered_pairs(user_input, covered_pairs, all_possible_pairs)

# Display suggestion and coverage status
suggestion = suggest_test_case(categories, covered_pairs, all_possible_pairs)
print("Suggested test case to maximally increase pair-wise coverage:", suggestion['suggested_test_case'])
print("Total missing pairs:", suggestion['total_missing_pairs'])
print("Total possible pairs:", suggestion['total_pairs'])
